In [ ]:
# stdlib
import asyncio
import time

# third party
import veilid

# first party
from veilid_streamer import VEILID_STREAMER_STREAM_PREFIX
from veilid_streamer import VeilidStreamer

In [ ]:
host = "localhost"
port = 5959

In [ ]:
async def veilid_callback(
    update: veilid.VeilidUpdate, app_message_queue: asyncio.Queue
):
    if update.kind in {
        veilid.VeilidUpdateKind.APP_MESSAGE,
        veilid.VeilidUpdateKind.APP_CALL,
    }:
        await app_message_queue.put(update)


app_message_queue = asyncio.Queue()
conn = await veilid.json_api_connect(
    host, port, lambda update: veilid_callback(update, app_message_queue)
)

In [ ]:
router = await (await conn.new_routing_context()).with_default_safety()
record = await router.create_dht_record(veilid.DHTSchema.dflt(1))
public_key, private_key = record.owner, record.owner_secret
await router.close_dht_record(record.key)
key_pair = veilid.KeyPair.from_parts(key=public_key, secret=private_key)
record_open = await router.open_dht_record(record.key, key_pair)
route_id, blob = await conn.new_custom_private_route(
    [veilid.CryptoKind.CRYPTO_KIND_VLD0],
    veilid.Stability.RELIABLE,
    veilid.Sequencing.ENSURE_ORDERED,
)
await router.set_dht_value(record_open.key, 0, blob)
self_remote_private_route = await conn.import_remote_private_route(blob)

In [ ]:
await router.app_message(self_remote_private_route, b"READY")
update = await app_message_queue.get()
assert update.detail.message == b"READY"
print(f"Your DHT Key: {record.key}")

In [ ]:
vs = VeilidStreamer(connection=conn, router=router)
while True:
    update = await app_message_queue.get()
    if update.detail.message.startswith(VEILID_STREAMER_STREAM_PREFIX):
        message = await vs.receive_stream(update)
        if message:
            start_time = int.from_bytes(message[:4], byteorder="big")
            print(
                f"Received {len(message)} bytes in {int(time.time()) - start_time} seconds"
            )